In [1]:
import pandas as pd
import os
# import seaborn as sns
import numpy as np
import re
from datetime import datetime
import seaborn as sns
path = './' + 'data/'
os.listdir(path)
# os.cpu_count()

['OD集货量',
 '中转场历史到件量0601至0831.csv',
 '场地数据.csv',
 '班次数据0601至0831.csv',
 '运力配载数据0601至0831']

In [2]:
# # %%time
# # trans = pd.read_json('./data/运力配载数据0601至0831/line_info_0601.json', lines=True)
# hist_arr = pd.read_csv(path + '中转场历史到件量0601至0831.csv')
# # loc = pd.read_csv(path + '场地数据.csv')
# # # loc.to_csv(path + '场地数据.csv', index = None, encoding='utf_8_sig')
# # task_arrange = pd.read_csv(path + '班次数据0601至0831.csv')


# path_od = path + 'OD集货量/'  
# # path+
# dfs = pd.DataFrame()
# for fname in os.listdir(path_od):
#     if re.search(r'\.csv$', fname):
#         dfs = pd.concat(
#             [dfs , pd.read_csv(path_od + fname)], axis = 0, ignore_index = True)
# prc_in = pd.DataFrame(dfs) 
# print(
#     'loc:', loc.shape, '\nhist:' ,
#     hist_arr.shape, '\ntask:',
#     task_arrange.shape, '\nod:' ,
#     dfs.shape)

In [ ]:
task_loc = pd.read_pickle('./data_clean/task_loc.p')

---

# Loc table clean  

In [4]:
loc = pd.read_pickle('./data_clean/loc.p'); loc.shape

(444, 21)

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
loc[loc['场地代码'] == '552W']

,场地代码,场地类型,主服务业务区,单小时处理能力,卸货卡位总数,卸车卡位（17.5M）,卸车卡位（>=15M）,卸车卡位（<15M）,装货卡位总数,装车卡位（17.5M）,装车卡位（15M）,装车卡位（13.5M）,装车卡位（9.6M）,装车卡位（7.6M）,装车卡位（5.2M）,off,onn,off_p1,off_p2,onn_p1,onn_p2
205,552W,片区中转场,551Y,21516,12,False,True,True,87,False,True,True,False,False,False,22.77636,28.5,1.89803,0.261797,2.375,0.327586


In [ ]:
# loc.columns
loc = loc.drop(['所在城市代码','应用场景', '分拨区' , '所在城市名称', '场地名称', '主服务城市','所在区部'], axis = 1) 
loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
       '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']]  = loc[[ '卸车卡位（17.5M）', '卸车卡位（>=15M）', '卸车卡位（<15M）', '装车卡位（17.5M）', '装车卡位（15M）',
       '装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']] == 0
# loc[['装车卡位（13.5M）', '装车卡位（9.6M）', '装车卡位（7.6M）', '装车卡位（5.2M）']].describe()
# loc.head(2)
loc['off_1'] = loc['卸车卡位（17.5M）'] * 17.5
loc['off_2'] = loc['卸车卡位（>=15M）'] * 15
loc['off_3'] = loc['卸车卡位（<15M）'] * (13.5 * 0.970721 + 9.6 * 11.056306 + 7.6 * 2.279279 + 5.2 * 9.826577)/ \
    (0.970721 + 11.056306 + 2.279279 + 9.826577)
loc['onn_1'] = loc['装车卡位（17.5M）'] * 17.5
loc['onn_2'] = loc['装车卡位（15M）'] * 15
loc['onn_3'] = loc['装车卡位（13.5M）'] * 13.5
loc['onn_4'] = loc['装车卡位（9.6M）'] * 9.6
loc['onn_5'] = loc['装车卡位（7.6M）'] * 7.6
loc['onn_6'] = loc['装车卡位（5.2M）'] * 5.2
loc['off'] = loc['off_1'] + loc['off_2'] + loc['off_3']
loc['onn'] = loc['onn_1'] + loc['onn_2'] + loc['onn_3'] + loc['onn_4'] + loc['onn_5'] + loc['onn_6']
loc = loc.drop(['off_1','off_2', 'off_3', 'onn_1', 'onn_2', 'onn_3', 'onn_4', 'onn_5', 'onn_6'], axis = 1)

loc['off_p1'] = loc['off']/loc['卸货卡位总数']
loc['off_p2'] = loc['off']/loc['装货卡位总数']
loc['onn_p1'] = loc['onn']/loc['卸货卡位总数']
loc['onn_p2'] = loc['onn']/loc['装货卡位总数']

# loc.to_pickle('./data_clean/loc.p')


In [ ]:
corr_loc = loc.corr()
corr_loc.style.background_gradient(cmap='coolwarm', axis=None)

# Task arrage clean

In [ ]:
# task_arrange[(task_arrange['operate_zone_code'] == '552W') & (task_arrange['batch_date'] == '2021-06-04')]

In [6]:
# pd.to_pickle(ta,  './data_clean/task.p')
ta = pd.read_pickle('./data_clean/task.p')
# task_loc[task_loc['operate_zone_code'] == '010R']

In [ ]:
%%time

ta = task_arrange.groupby(['operate_zone_code', 'batch_date']).\
    agg(min_begin = ('plan_begin_tm', min),
    max_begin = ('plan_begin_tm', max),
    min_end = ('plan_end_tm', min),
    max_end = ('plan_end_tm', max),
    count = ('batch_code', 'nunique')
    
    ).reset_index()

def convt_min(col):
    ta[col] = ta[col].apply(lambda i: datetime.strptime(i,'%H:%M').hour*60 + datetime.strptime(i,'%H:%M').minute)
     
for i in ['min_begin', 'max_begin', 'min_end','max_end']:
    convt_min(i)


# ta['time_consume_per_vote'] = (ta['max_end'] - ta['min_begin'])/ta['count']

In [ ]:
corr_ta = ta.corr()
corr_ta.style.background_gradient(cmap='coolwarm', axis=None)


In [31]:
# %%time
# import matplotlib.pyplot as plt
# ta_plt2 = pd.DataFrame(ta['operate_zone_code'].value_counts())
# ta_plt2 = pd.DataFrame(ta_plt2['operate_zone_code'].value_counts())
# ta_plt2['site'] = ta_plt2.index
# plt.pie(x = 'operate_zone_code' , data =ta_plt2)

In [32]:
%%time
task_loc = ta.merge(loc, left_on='operate_zone_code', right_on = '场地代码', how = 'left')

Wall time: 142 ms


In [ ]:
%%time
del loc, ta 
# task_loc.to_pickle('./data_clean/task_loc.p')


In [33]:
%%time
task_loc_valued = task_loc[~task_loc['场地代码'].isna()]
task_loc_nulled = task_loc[task_loc['场地代码'].isna()]

In [ ]:
# task_loc.isnull().sum()/len(task_loc)

---
# od

In [ ]:
[prc_in[(prc_in['date1'] == '2021-06-08') & (prc_in['destsitecode'] == '552W')].sum(),
prc_in[(prc_in['date1'] == '2021-06-04') & (prc_in['srcsitecode'] == '552W')].sum()]

In [ ]:
# prc_in.head()

od = prc_in.groupby(['srcsitecode','destsitecode', 'date1']).agg(
    batch_cnt = ('srcbatchcode', 'nunique'),
    route_cnt = ('route_code', 'nunique'),
    votes = ('votes', sum),
    weights = ('weight1', sum),
    air_votes =('air_votes', sum),
    air_weights = ('air_weight1', sum)
).reset_index()

In [ ]:
# od.to_pickle('./data_clean/od.p')

In [ ]:
# hist_arr[hist_arr['a.report_dt'] == '2021-06-01'].sort_values('a.zonecode')

In [ ]:
od

In [ ]:
# test  =prc_in[(prc_in['date1'] == '2021-06-01') & (prc_in['srcbatchcode'] == '010R2100')]

In [ ]:
od.columns

In [ ]:
od_src = od[['srcsitecode','date1', 'batch_cnt', 'route_cnt',
       'votes', 'weights', 'air_votes', 'air_weights']]

In [ ]:
od.head()

---

# hist

In [ ]:
hist_arr[(hist_arr['a.zonecode'] == '552W') &  ( hist_arr['a.report_dt'] == '2021-06-04' )]

# hist_arr.to_pickle('./data_clean/hist_arr.p')
# merge all

In [ ]:
hist_arr.merge(od, left_on = ['a.report_dt', 'a.zonecode'], right_on = '')

In [ ]:
task_loc.merge(hist_arr, \
    left_on = ['operate_zone_code', 'batch_date'], right_on=['a.zonecode', 'a.report_dt'], validate = 'one_to_one',
    how = 'right')

In [ ]:
[len(task_loc), len(train), len(od)]

In [ ]:
# od

---
# transport


In [ ]:
trans.head(2)

In [ ]:
trans.iloc[11,:]['passZoneCodes'][0]

In [ ]:
trans.iloc[11,:]['taskInfos'][0] # ['unloadZone']

In [ ]:
trans.iloc[1,:]['vehicleInfos'][0] # ['unloadZone']